In [42]:
import pandas as pd
import pandasql
import sqlite3

In [2]:
account_data = pd.read_excel('data/bank_account_data.xlsx')
account_data.head()

,Cust_ID,Purchase,Cheq_Bal,Sav_Bal,Debt_Bal,Joint_Acct,Age,Date
0,1,0,1739.48,26918.18,12563.93,0,49,NaT
1,2,0,2862.35,23093.67,15457.88,0,39,NaT
2,3,0,2296.66,18037.33,12721.89,0,52,NaT
3,4,1,2880.28,32317.63,7075.03,0,28,2017-04-30
4,5,0,998.27,15110.04,11356.13,0,43,NaT


In [25]:
#Опишите имеющиеся колонки в данных - сделайте аналог describe в pandas для колонок.
cust_id_data = pandasql.sqldf("SELECT COUNT(cust_id), AVG(cust_id), MIN(cust_id), MAX(cust_id) FROM account_data;")
print(cust_id_data)
#cust_id - значения от 1 до 200, пропусков нет. Представляет собой уникальный идентификатор записи в датасете. 

purchase_data = pandasql.sqldf("SELECT COUNT(purchase), AVG(purchase), MIN(purchase), MAX(purchase) FROM account_data;")
print(purchase_data)
#purchase - значения 0 и 1. Признак покупки ипотеки клиентом.

cheq_bal_data = pandasql.sqldf("SELECT COUNT(cheq_bal), AVG(cheq_bal), MIN(cheq_bal), MAX(cheq_bal) FROM account_data;")
print(cheq_bal_data)
#cheq_bal - значения от 644.31 до 4437.44, среднее 2145.09, пропусков нет. Один из видов баланса на счету клиента.

sav_bal_data = pandasql.sqldf("SELECT COUNT(sav_bal), AVG(sav_bal), MIN(sav_bal), MAX(sav_bal) FROM account_data;")
print(sav_bal_data)
#sav_bal - значения от 6558.91 до 39132.63, среднее 22084.43, пропусков нет. Один из видов баланса на счету клиента.

debt_bal_data = pandasql.sqldf("SELECT COUNT(debt_bal), AVG(debt_bal), MIN(debt_bal), MAX(debt_bal) FROM account_data;")
print(debt_bal_data)
#debt_bal - значения от 0 до 20559.7, среднее 8638.37, пропусков нет. Один из видов баланса на счету клиента.

joint_acct_data = pandasql.sqldf("SELECT COUNT(joint_acct), AVG(joint_acct), MIN(joint_acct), MAX(joint_acct) FROM account_data;")
print(joint_acct_data)
#joint_acct - значения 0 и 1. Признак объединенного счета.

age_data = pandasql.sqldf("SELECT COUNT(age), AVG(age), MIN(age), MAX(age) FROM account_data;")
print(age_data)
#age - значения от 18 до 60, среднее 38.5, пропусков нет. Возраст клиента.

date_data = pandasql.sqldf("SELECT COUNT(date), MIN(date), MAX(date) FROM account_data;")
print(date_data)
#date - значения от 01.04.2017 до 29.08.2017, заполнена только для 39 записей. Дата получения ипотеки.

   COUNT(cust_id)  AVG(cust_id)  MIN(cust_id)  MAX(cust_id)
0             200         100.5             1           200
   COUNT(purchase)  AVG(purchase)  MIN(purchase)  MAX(purchase)
0              200          0.195              0              1
   COUNT(cheq_bal)  AVG(cheq_bal)  MIN(cheq_bal)  MAX(cheq_bal)
0              200      2145.0935         644.31        4437.44
   COUNT(sav_bal)  AVG(sav_bal)  MIN(sav_bal)  MAX(sav_bal)
0             200   22084.42895       6558.91      39132.63
   COUNT(debt_bal)  AVG(debt_bal)  MIN(debt_bal)  MAX(debt_bal)
0              200      8638.3696            0.0        20559.7
   COUNT(joint_acct)  AVG(joint_acct)  MIN(joint_acct)  MAX(joint_acct)
0                200            0.295                0                1
   COUNT(age)  AVG(age)  MIN(age)  MAX(age)
0         200    38.535        18        60
   COUNT(date)                   MIN(date)                   MAX(date)
0           39  2017-04-01 00:00:00.000000  2017-08-29 00:00:00.000000


In [29]:
#Посчитайте совокупный объем средств у клиентов в банке - сумма по всем балансам.
bal_data = pandasql.sqldf("SELECT SUM(cheq_bal) + SUM(sav_bal) + SUM(debt_bal) bal_sum FROM account_data;")
print(bal_data)
#Совокупный объем средств: 6'573'578.41

      bal_sum
0  6573578.41


In [28]:
#Выведите только тех клиентов, которые приобрели ипотеку (колонка mortgage).
#Видимо, имеется в виду колонка purchase, т.к. колонки mortgage в датасете нет.
mortgage_data = pandasql.sqldf(
    """
    SELECT cust_id
    FROM account_data
    WHERE purchase = 1
    """)
print(mortgage_data)

    Cust_ID
0         4
1         6
2         7
3        11
4        13
5        17
6        18
7        20
8        22
9        31
10       36
11       38
12       40
13       41
14       54
15       56
16       59
17       60
18       74
19       76
20       80
21       87
22       91
23       93
24       94
25       97
26      100
27      107
28      111
29      113
30      132
31      146
32      154
33      166
34      169
35      182
36      184
37      185
38      189


In [33]:
#Какой совокупный объем средств в среднем у клиентов, которые приобрели ипотеку и у клиентов, которые её не приобретали?
clients_with_mortgage = pandasql.sqldf(
    """
    SELECT AVG(cheq_bal) + AVG(sav_bal) + AVG(debt_bal) bal_avg
    FROM account_data
    WHERE purchase = 1
    """)
print(clients_with_mortgage)

clients_without_mortgage = pandasql.sqldf(
    """
    SELECT AVG(cheq_bal) + AVG(sav_bal) + AVG(debt_bal) bal_avg
    FROM account_data
    WHERE purchase = 0
    """)
print(clients_without_mortgage)

#Клиенты с ипотекой: 36054.9
#Клиенты без ипотеки: 32095.89

        bal_avg
0  36054.895385
        bal_avg
0  32095.885031


In [35]:
#Есть ли разница в среднем по каждому из типов баланса у клиентов, которые приобрели ипотеку и у клиентов, которые её не приобретали?
clients_with_mortgage = pandasql.sqldf(
    """
    SELECT
        AVG(cheq_bal) cheq_bal_avg,
        AVG(sav_bal) sav_bal_avg,
        AVG(debt_bal) debt_bal_avg
    FROM account_data
    WHERE purchase = 1
    """)
print(clients_with_mortgage)

clients_without_mortgage = pandasql.sqldf(
    """
    SELECT
        AVG(cheq_bal) cheq_bal_avg,
        AVG(sav_bal) sav_bal_avg,
        AVG(debt_bal) debt_bal_avg
    FROM account_data
    WHERE purchase = 0
    """)
print(clients_without_mortgage)

#У клиентов без ипотеки баланс debt_bal, в среднем, почти в 3 раза выше, чем у клиентов с ипотекой.
#Балансы cheq_bal и sav_bal выше у клиентов с ипотекой.

   cheq_bal_avg   sav_bal_avg  debt_bal_avg
0   2757.975641  29627.564359   3669.355385
   cheq_bal_avg   sav_bal_avg  debt_bal_avg
0   1996.631366  20257.209814   9842.043851


In [41]:
#Выберите самого возрастного получателя ипотеки.
mortgage_data = pandasql.sqldf(
    """
    SELECT
       cust_id,
       age
    FROM account_data
    WHERE purchase = 1
    ORDER BY age DESC
    LIMIT 1
    """)
print(mortgage_data)
#Самый возрастной получатель ипотеки имеет возраст 45 лет, ID записи: 41.

   Cust_ID  Age
0       41   45


In [39]:
#Выберите клиента, который раньше всех получил ипотеку.
mortgage_data = pandasql.sqldf(
    """
    SELECT
       cust_id,
       date
    FROM account_data
    WHERE purchase = 1
    ORDER BY date
    LIMIT 1
    """)
print(mortgage_data)
#Раньше всех получил ипотеку клиент с ID = 185. Дата получения ипотеки: 01.04.2017

   Cust_ID                        Date
0      185  2017-04-01 00:00:00.000000


In [48]:
#Создайте любую базу данных в sqlite3, состоящую из 5 колонок, одна из колонок обязательно должна быть типа REAL. (Названия колонок любые, какие захотите)
conn = sqlite3.connect('data/hw1.db')
conn.execute('''
CREATE TABLE countries (
    id INTEGER, 
    name STRING,
    capital STRING, 
    population INTEGER, 
    avg_life_expectancy REAL
    );''')

conn.commit()
conn.close()

In [50]:
#Добавьте 3 полностью заполненные строки в вашу БД.
conn = sqlite3.connect('data/hw1.db')

conn.execute("INSERT INTO countries VALUES ('1', 'Россия', 'Москва', '147', '71.34');")
conn.execute("INSERT INTO countries VALUES ('2', 'США', 'Вашингтон', '333', '77.28');")
conn.execute("INSERT INTO countries VALUES ('3', 'Китай', 'Москва', '1412', '71.1');")

conn.commit()
conn.close()

In [59]:
#Удалите строку из колонки типа REAL (из пункта 8), в которой значение признака среднее (то есть не минимальное и не максимальное).
#Напрямую указывать значение из этой строки нельзя.
conn = sqlite3.connect('data/hw1.db')

conn.execute("""
    DELETE FROM countries
    WHERE id IN (
        SELECT id
        FROM countries
        ORDER BY avg_life_expectancy
        LIMIT 1 OFFSET 1)""")

conn.commit()
conn.close()

In [60]:
conn = sqlite3.connect('data/hw1.db')
cur = conn.cursor()
cur.execute("SELECT * from countries;")
result = cur.fetchall()
conn.close()
result

[(2, 'США', 'Вашингтон', 333, 77.28), (3, 'Китай', 'Москва', 1412, 71.1)]